In [44]:
!pip install pyodbc pandas

import pyodbc
import pandas as pd


In [45]:
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=REHAN;'  # ✅ This is your instance name
    'DATABASE=gravity_books;'
    'Trusted_Connection=yes;'
)

print("✅ Connected successfully!")


✅ Connected successfully!


In [46]:
df_author = pd.read_sql("SELECT * FROM author", conn)
df_publisher = pd.read_sql("SELECT * FROM publisher", conn)
df_book_language = pd.read_sql("SELECT * FROM book_language", conn)
df_book = pd.read_sql("SELECT * FROM book", conn)
df_book_author = pd.read_sql("SELECT * FROM book_author", conn)
df_address_status = pd.read_sql("SELECT * FROM address_status", conn)
df_country = pd.read_sql("SELECT * FROM country", conn)
df_address = pd.read_sql("SELECT * FROM address", conn)
df_customer = pd.read_sql("SELECT * FROM customer", conn)
df_customer_address = pd.read_sql("SELECT * FROM customer_address", conn)
df_shipping_method = pd.read_sql("SELECT * FROM shipping_method", conn)
df_cust_order = pd.read_sql("SELECT * FROM cust_order", conn)
df_order_status = pd.read_sql("SELECT * FROM order_status", conn)
df_order_line = pd.read_sql("SELECT * FROM order_line", conn)
df_order_history = pd.read_sql("SELECT * FROM order_history", conn)



C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\750228678.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_author = pd.read_sql("SELECT * FROM author", conn)
C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\750228678.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_publisher = pd.read_sql("SELECT * FROM publisher", conn)
C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\750228678.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_book_language = pd.read_sql("SELECT * FROM book_language", conn)
C:\Users\sa

In [48]:
# Check missing values
print(df_book.isnull().sum())
print(df_customer.isnull().sum())

# Fill or drop missing values
df_book.fillna({'num_pages': 0}, inplace=True)
df_customer.dropna(subset=['email'], inplace=True)


book_id             0
title               0
isbn13              0
language_id         0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64
customer_id    0
first_name     0
last_name      0
email          0
dtype: int64


In [60]:
# Merge orders with customers
df_order_customer = pd.merge(df_cust_order, df_customer, on='customer_id', how='left')

# Merge order lines (books in order)
df_order_full = pd.merge(df_order_customer, df_order_line, on='order_id', how='left')

# Add book info
df_order_full = pd.merge(df_order_full, df_book, on='book_id', how='left')

# Add publisher info
df_order_full = pd.merge(df_order_full, df_publisher, on='publisher_id', how='left')

# Add language info
df_order_full = pd.merge(df_order_full, df_book_language, on='language_id', how='left')

df_order_full.head()


,order_id,order_date,customer_id,shipping_method_id,dest_address_id,first_name,last_name,email,churn,line_id,...,price,title,isbn13,language_id,num_pages,publication_date,publisher_id,publisher_name,language_code,language_name
0,1,2024-06-01 10:00:00,1,1,1,John,Doe,john@example.com,1,1.0,...,20.99,Harry Potter,9780747532743,1.0,320.0,1997-06-26,1.0,Penguin Random House,EN,English
1,2,2024-06-02 14:30:00,2,2,2,Aisha,Khan,aisha@example.com,1,2.0,...,15.49,1984,9780451524935,1.0,328.0,1949-06-08,2.0,HarperCollins,EN,English
2,201,2025-05-25 11:23:21,101,1,1,Alice,Sharma,alice@example.com,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202,2024-11-26 11:23:21,102,1,1,Bob,Kumar,bob@example.com,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,203,2024-05-20 11:23:21,103,1,1,Charlie,Singh,charlie@example.com,1,3.0,...,450.00,Data Science 101,9781234567890,1.0,250.0,2023-01-01,1.0,Penguin Random House,EN,English


In [50]:
# See missing values
df_order_full.isnull().sum()

# Example: fill missing page count or drop rows if needed
df_order_full['num_pages'].fillna(0, inplace=True)

# Drop rows with critical nulls (if any)
df_order_full.dropna(subset=['title', 'price'], inplace=True)


C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\1258638551.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_order_full['num_pages'].fillna(0, inplace=True)


In [51]:
# Convert order_date to datetime
df_order_full['order_date'] = pd.to_datetime(df_order_full['order_date'])

# Create month, year, day features
df_order_full['order_month'] = df_order_full['order_date'].dt.month
df_order_full['order_year'] = df_order_full['order_date'].dt.year

# Calculate book age
df_order_full['book_age'] = 2025 - pd.to_datetime(df_order_full['publication_date']).dt.year

# Total price (you can also include quantity if you add that column later)
df_order_full['total_price'] = df_order_full['price']  # if quantity exists: * quantity


In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode publisher and language
df_order_full['publisher_enc'] = le.fit_transform(df_order_full['publisher_name'])
df_order_full['language_enc'] = le.fit_transform(df_order_full['language_name'])

# Optionally encode customer info like name initials or regions later


In [53]:
df_order_full[['customer_id', 'title', 'book_age', 'order_month', 'price', 'publisher_enc', 'language_enc']].head()


,customer_id,title,book_age,order_month,price,publisher_enc,language_enc
0,1,Harry Potter,28,6,20.99,1,0
1,2,1984,76,6,15.49,0,0
4,103,Data Science 101,2,5,450.00,1,0
5,104,Python Deep Dive,3,6,399.00,1,0
6,105,Data Science 101,2,6,450.00,1,0


In [54]:
import pandas as pd
from datetime import datetime, timedelta

# Convert order_date to datetime
df_cust_order['order_date'] = pd.to_datetime(df_cust_order['order_date'])

# Define churn cutoff (6 months before latest order date)
cutoff_date = df_cust_order['order_date'].max() - pd.DateOffset(months=6)

# Find last order date per customer
last_order = df_cust_order.groupby('customer_id')['order_date'].max().reset_index()
last_order['churn'] = last_order['order_date'] < cutoff_date

# Merge churn label into main customer dataset
df_customer = pd.merge(df_customer, last_order[['customer_id', 'churn']], on='customer_id', how='left')

# Convert True/False to int (1 for churn, 0 for active)
df_customer['churn'] = df_customer['churn'].fillna(True).astype(int)

df_customer.head()


C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\3145565349.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_customer['churn'] = df_customer['churn'].fillna(True).astype(int)


,customer_id,first_name,last_name,email,churn
0,1,John,Doe,john@example.com,1
1,2,Aisha,Khan,aisha@example.com,1
2,101,Alice,Sharma,alice@example.com,0
3,102,Bob,Kumar,bob@example.com,1
4,103,Charlie,Singh,charlie@example.com,1


In [55]:
# Total number of orders per customer
order_counts = df_cust_order.groupby('customer_id')['order_id'].count().reset_index(name='num_orders')

# Average order value (joining with order_line table)
order_line_value = df_order_line.groupby('order_id')['price'].sum().reset_index(name='order_total')
df_order_value = pd.merge(df_cust_order, order_line_value, on='order_id')
avg_order_value = df_order_value.groupby('customer_id')['order_total'].mean().reset_index(name='avg_order_value')

# Days since last order
today = pd.to_datetime('today')
last_order['days_since_last_order'] = (today - last_order['order_date']).dt.days

# Merge all features
features = df_customer[['customer_id', 'churn']]
features = features.merge(order_counts, on='customer_id', how='left')
features = features.merge(avg_order_value, on='customer_id', how='left')
features = features.merge(last_order[['customer_id', 'days_since_last_order']], on='customer_id', how='left')

# Fill NaNs with 0 (in case of missing customers)
features.fillna(0, inplace=True)

features.head()


,customer_id,churn,num_orders,avg_order_value,days_since_last_order
0,1,1,1.0,20.99,389.0
1,2,1,1.0,15.49,387.0
2,101,0,1.0,0.00,30.0
3,102,1,1.0,0.00,210.0
4,103,1,1.0,450.00,400.0


In [56]:
from sklearn.model_selection import train_test_split

X = features[['num_orders', 'avg_order_value', 'days_since_last_order']]
y = features['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [15]:
!pip install notebook

  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
Using cached webcolors-24.11.1-py3-none-any.whl (14 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)


In [57]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define model
model = Sequential()
model.add(Dense(16, input_dim=3, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)


C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3750 - loss: 19.9895
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.7500 - loss: 19.3829
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.7500 - loss: 18.7834
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.7500 - loss: 18.1926
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.7500 - loss: 17.6034
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.7500 - loss: 17.0145
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.7500 - loss: 16.4265
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.7500 - loss: 15.8593
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.7500 - loss: 15.3507
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7500 - loss: 14.8480
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7500 - loss: 14.3736
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.75

In [58]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Results
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
[[0 1]
 [1 2]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      0.67      0.67         3

    accuracy                           0.50         4
   macro avg       0.33      0.33      0.33         4
weighted avg       0.50      0.50      0.50         4



In [61]:
# GENRE PREDICTION MODEL (Assuming you have added 'genre' column in book table)

import pandas as pd
import numpy as np
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

# Connect to SQL Server
db_conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=REHAN;'
    'DATABASE=gravity_books;'
    'Trusted_Connection=yes;'
)

# Load required tables into DataFrame
query = '''
SELECT b.book_id, b.title, b.language_id, b.num_pages, b.publisher_id, b.genre,
       p.publisher_name, bl.language_name
FROM book b
JOIN publisher p ON b.publisher_id = p.publisher_id
JOIN book_language bl ON b.language_id = bl.language_id
WHERE b.genre IS NOT NULL
'''



In [68]:
df_genre = pd.read_sql(query, db_conn)

# Encode categorical features
le_genre = LabelEncoder()
df_genre['genre_encoded'] = le_genre.fit_transform(df_genre['genre'])

le_lang = LabelEncoder()
df_genre['language_encoded'] = le_lang.fit_transform(df_genre['language_name'])

le_pub = LabelEncoder()
df_genre['publisher_encoded'] = le_pub.fit_transform(df_genre['publisher_name'])

# Final feature set
X = df_genre[['num_pages', 'language_encoded', 'publisher_encoded']]
y = df_genre['genre_encoded']



C:\Users\sanam\AppData\Local\Temp\ipykernel_45004\3267038754.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_genre = pd.read_sql(query, db_conn)


In [69]:
# Build ANN model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(len(df_genre['genre_encoded'].unique()), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [70]:

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 18.8967 - val_accuracy: 0.0000e+00 - val_loss: 146.7666
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.5000 - loss: 18.4175 - val_accuracy: 0.0000e+00 - val_loss: 145.8034
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.5000 - loss: 17.9378 - val_accuracy: 0.0000e+00 - val_loss: 144.8506
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.5000 - loss: 17.4575 - val_accuracy: 0.0000e+00 - val_loss: 143.9034
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.5000 - loss: 16.9767 - val_accuracy: 0.0000e+00 - val_loss: 142.9590
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - accuracy: 0.5000 - loss: 16.4953 - val_accuracy: 0.0000e+00 - val_loss: 142.0160
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.5000 - loss: 16.0134 - val_accuracy: 0.0000e+00 - val_loss: 141.0733
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step - accuracy: 0.5000

In [72]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred_labels))
print(classification_report(y_test, y_pred_labels))


[[0 0 1]
 [2 0 1]
 [0 0 0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       3.0
           3       0.00      0.00      0.00       0.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0



C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\san

In [74]:
# Get only the classes that appear in y_test
import numpy as np
unique_labels = np.unique(y_test)

# Get matching class names
matched_names = [le_genre.classes_[i] for i in unique_labels]

print(classification_report(y_test, y_pred_labels, labels=unique_labels, target_names=matched_names))


                  precision    recall  f1-score   support

Computer Science       0.00      0.00      0.00       1.0
       Dystopian       0.00      0.00      0.00       3.0

       micro avg       0.00      0.00      0.00       4.0
       macro avg       0.00      0.00      0.00       4.0
    weighted avg       0.00      0.00      0.00       4.0



C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sanam\anaconda3\envs\tf_env\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [81]:
!pip install matplotlib
!pip install seaborn

In [82]:
import pandas as pd
import numpy as np

# Simulate user-book ratings
np.random.seed(42)
user_ids = np.arange(1, 21)       # 20 users
book_ids = np.arange(1, 16)       # 15 books

# Create random user-book-rating entries
ratings_data = {
    'user_id': np.random.choice(user_ids, 100),
    'book_id': np.random.choice(book_ids, 100),
    'rating': np.random.randint(1, 6, size=100)  # Ratings from 1 to 5
}

df_ratings = pd.DataFrame(ratings_data)
df_ratings.head()


,user_id,book_id,rating
0,7,3,1
1,20,12,2
2,15,8,4
3,11,6,4
4,8,11,2


In [83]:
from sklearn.metrics.pairwise import cosine_similarity

# Pivot to create a matrix: rows = users, columns = books
ratings_matrix = df_ratings.pivot_table(index='user_id', columns='book_id', values='rating')

# Fill NaNs with 0 (assume unrated books have 0 rating)
ratings_matrix_filled = ratings_matrix.fillna(0)
ratings_matrix_filled.head()


book_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
user_id,,,,,,,,,,,,,,,
1,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,1.0
2,2.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,4.0,4.0,4.0,0.0
3,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0


In [84]:
# Cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix_filled)

# Convert to DataFrame for readability
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix_filled.index, columns=ratings_matrix_filled.index)
user_similarity_df.head()


user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
user_id,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.130647,0.000000,0.184615,0.114274,0.130596,0.348890,0.198465,0.261746,0.461727,0.000000,0.231932,0.052009,0.000000,0.084441,0.531187,0.000000,0.474780,0.064158,0.123351
2,0.130647,1.000000,0.063420,0.634075,0.252310,0.027462,0.556349,0.876396,0.484706,0.252439,0.000000,0.529161,0.746420,0.312218,0.264125,0.384317,0.000000,0.611499,0.283315,0.363137
3,0.000000,0.063420,1.000000,0.084017,0.234023,0.000000,0.158777,0.189241,0.245921,0.120073,0.432136,0.295540,0.071007,0.753850,0.204951,0.065556,0.466041,0.064820,0.000000,0.000000
4,0.184615,0.634075,0.084017,1.000000,0.482808,0.000000,0.415761,0.589723,0.285387,0.068599,0.246885,0.241209,0.304256,0.315135,0.054887,0.374532,0.221880,0.462910,0.000000,0.427618
5,0.114274,0.252310,0.234023,0.482808,1.000000,0.000000,0.280745,0.205329,0.000000,0.000000,0.286534,0.139973,0.282494,0.487660,0.203844,0.608552,0.000000,0.257881,0.000000,0.595550


In [86]:
def recommend_books(user_id, num_recommendations=5):
    # Get similarity scores for this user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)
    similar_users = similar_users.drop(user_id)  # Remove self-similarity

    # Weighted average of ratings from top similar users
    weighted_ratings = pd.Series(dtype=float)

    for sim_user_id, similarity in similar_users.items():
        sim_user_ratings = ratings_matrix_filled.loc[sim_user_id]
        weighted_ratings = weighted_ratings.add(sim_user_ratings * similarity, fill_value=0)

    # Normalize by similarity sum
    similarity_sum = similar_users.sum()
    if similarity_sum > 0:
        weighted_ratings = weighted_ratings / similarity_sum

    # Filter out books the user has already rated
    user_rated_books = ratings_matrix.loc[user_id].dropna().index
    recommendations = weighted_ratings.drop(user_rated_books).sort_values(ascending=False)

    return recommendations.head(num_recommendations)


In [87]:
# Recommend 5 books for user_id = 3
recommended_books = recommend_books(user_id=3, num_recommendations=5)
print("Top Recommended Books for User 3:\n", recommended_books)


Top Recommended Books for User 3:
 book_id
7     1.409696
8     0.936858
12    0.819685
6     0.739804
13    0.688441
dtype: float64
